In [1]:
import flask
from flask import render_template

from flask import request

import tensorflow as tf
import pickle

In [ ]:
app = flask.Flask(__name__, template_folder = 'templates')

@app.route('/', methods = ['POST', 'GET'])

@app.route('/index', methods = ['POST', 'GET'])
def main():
    if flask.request.method == 'GET':
        return render_template('main.html')

    if flask.request.method == 'POST':
        param_list = ('Плотность, кг/м3', 'модуль упругости, ГПа', 'Количество отвердителя, м.%', 
                      'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2', 'Поверхностная плотность, г/м2	', 
                      'Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Потребление смолы, г/м2',
                      'Угол нашивки, град', 'Шаг нашивки', 'Плотность нашивки')
        params_html = []
        params = []
        for i in param_list:
            param = request.form.get(i)
            params.append(param)
            params_html.append(param)
        params = [float(i.replace(',', '.')) for i in params]
        
        loaded_minmax = pickle.load(open('models/scaler_norm_X.sav', 'rb'))
        result_input = loaded_minmax.transform([params])
        
        model_pred = tf.keras.models.load_model('models/model_saved')
        result_pred = model_pred.predict(result_input)
        
        loaded_inv = pickle.load(open('models/scaler_norm_y.sav', 'rb'))
        result_output = loaded_inv.inverse_transform(result_pred)
                
        message = f'Соотношение матрица-наполнитель: {str(round(result_output[0][0], 4))}'

        return render_template('main.html', message = message, params = params_html)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
